# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [32]:
from tchub.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [33]:
config = AttrDict(
    use_wandb = True, # Whether to use or not wandb for experiment tracking
    wandb_group = None, # Whether to group this run in a wandb group
    wandb_entity = 'pacmel',
    wandb_project = 'tchub',
    enc_artifact = 'pacmel/tchub/mvp:run-tchub-8eyw9zfl', # Name:version of the encoder artifact
    input_ar = None,
    cpu = False
)

## Run

In [34]:
run = wandb.init(entity=config.wandb_entity,
                    project=config.wandb_project if config.use_wandb else 'work-nbs', 
                    group=config.wandb_group,
                    job_type='embeddings', 
                    mode='online' if config.use_wandb else 'disabled',
                    anonymous = 'never' if config.use_wandb else 'must',
                    config=config,
                    #id = 'embeddingsProvider',
                    resume='allow')

wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [35]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [36]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')


InvalidVersionSpec: Invalid version '1.1.10<2.0.0': invalid character(s)



In [37]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
enc_learner

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [38]:
# TODO: Correct order??
# 1 is the training artifact, 0 the validation
enc_run = enc_artifact.logged_by()
uas = enc_run.used_artifacts()
enc_artifact_train = uas[1] if len(uas) > 1 else uas[0] 
enc_artifact_valid = uas[0]
enc_artifact_train.name, enc_artifact_valid.name

('taxi:v0', 'taxi:v0')

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [39]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_valid.entity}/{enc_artifact_valid.project}/{enc_artifact_valid.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'taxi:v0'

Now we need to check whether the artifact that is going to be used fort the dimensionality reduction matches the artifact used to train the encoder. Matching means having the same number of variables, the same window size and stride, and the same frequency.

In [40]:
#export
def check_compatibility(input_ar:TSArtifact, enc_ar:TSArtifact):
    "Function to check that the artifact used to train the encoder model and the artifact that is \
    going to be passed to do inference"
    try:
        # Check that both artifacts have the same variables
        chk_vars = input_ar.metadata['TS']['vars'] == enc_ar.metadata['TS']['vars']
        # Check that both artifacts have the same freq
        chk_freq = input_ar.metadata['TS']['freq'] == enc_ar.metadata['TS']['freq']
        # Check that the dr artifact is not normalized (not normalized data has not the key normalization)
        chk_norm = input_ar.metadata['TS'].get('normalization') is None
        # Check that the dr artifact has not missing values
        chk_miss = input_ar.metadata['TS']['has_missing_values'] == "False"
        # Check all logical vars.
        if chk_vars and chk_freq and chk_norm and chk_miss:
            print("Artifacts are compatible.")
        else:
            raise Exception
    except Exception as e:
        print("Artifacts are not compatible.")
        raise e
    return None


In [41]:
df = input_ar.to_df()
df

,value
timestamp,
2014-10-01 00:00:00,12751
2014-10-01 00:30:00,8767
2014-10-01 01:00:00,7005
2014-10-01 01:30:00,5257
2014-10-01 02:00:00,4189
...,...
2014-12-14 21:30:00,16344
2014-12-14 22:00:00,15913
2014-12-14 22:30:00,14327


In [42]:
df.shape

(3600, 1)

In [43]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(3553, 1, 48)

In [44]:
%%time
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

/home/victor/env/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


CPU times: user 1.63 s, sys: 3.68 ms, total: 1.64 s
Wall time: 1.63 s


In [45]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings'), 
                     aliases=f'run-{run.project}-{run.id}')

In [46]:
run.finish()